# Get Started with Milvus

This notebook shows how to get started with [Milvus vector database](https://milvus.io/).

## Start Milvus DB (optional)

Milvus is available through the [Zilliz Cloud for Azure](https://zilliz.com/blog/zilliz-cloud-now-available-on-microsoft-azure) managed offering or [Azure Container Apps Add-Ons](https://learn.microsoft.com/azure/container-apps/services).

If you don't already have an instance of Milvus, you can run a local version for dev/test scenarios.

In [1]:
docker-compose up -d

Creating network "milvus" with the default driver
Pulling etcd (quay.io/coreos/etcd:v3.5.5)...
v3.5.5: Pulling from coreos/etcd
Digest: sha256:89b6debd43502d1088f3e02f39442fd3e951aa52bee846ed601cf4477114b89e
Status: Downloaded newer image for quay.io/coreos/etcd:v3.5.5
Pulling minio (minio/minio:RELEASE.2023-03-20T20-16-18Z)...
RELEASE.2023-03-20T20-16-18Z: Pulling from minio/minio
Digest: sha256:6d770d7f255cda1f18d841ffc4365cb7e0d237f6af6a15fcdb587480cd7c3b93
Status: Downloaded newer image for minio/minio:RELEASE.2023-03-20T20-16-18Z
Pulling standalone (milvusdb/milvus:v2.2.14)...
v2.2.14: Pulling from milvusdb/milvus
Digest: sha256:099bd246ae15242eeb6e4ba9263b977eb9d92d069f28b3f51e1a4986cab0f90d
Status: Downloaded newer image for milvusdb/milvus:v2.2.14
Creating milvus-etcd ... 
Creating milvus-minio ... 
Creating milvus-minio ... done
Creating milvus-etcd  ... done
Creating milvus-standalone ... 
Creating milvus-standalone ... done


## Install Milvus C# SDK

In [2]:
#r "nuget: Milvus.Client, 2.2.2-preview.6"

Installed Packages Milvus.Client, 2.2.2-preview.6

In [3]:
using Milvus.Client;

## Initialize Milvus Client

NOTE: If you have an instance of Milvus deployed in the cloud or your datacenter, replace `localhost` with your instance's host.

In [4]:
var milvusClient = new MilvusClient("localhost");

## Cleanup (optional)

In [5]:
await milvusClient.GetCollection("movies").DropAsync()

## List collections

In [6]:
var collections = milvusClient.GetCollection("movies");

In [7]:
collections

Name,movies


## Create collection

### Define schema

In [8]:
var schema = new CollectionSchema
{
    Fields =
    {
        FieldSchema.Create<long>("movie_id", isPrimaryKey: true),
        FieldSchema.CreateVarchar("movie_name", maxLength: 200),
        FieldSchema.CreateFloatVector("movie_description", dimension: 2)
    },
    Description = "Test movie search",
    EnableDynamicFields = true
};

### Create collection

In [9]:
var collection = await milvusClient.CreateCollectionAsync(collectionName: "movies",schema: schema, shardsNum: 2);

In [10]:
await collection.DescribeAsync()

Milvus.Client.MilvusCollectionDescription Aliases [ ] CollectionName movies CollectionId 447879535187460250 ConsistencyLevel Session CreationTimestamp 1708524868330 Schema Milvus.Client.CollectionSchema Name movies Description Test movie search Fields index value 0 Milvus.Client.FieldSchema Name movie_id DataType Int64 IsPrimaryKey True AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension <null> State FieldCreated FieldId 100 1 Milvus.Client.FieldSchema Name movie_name DataType VarChar IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength 200 Dimension <null> State FieldCreated FieldId 101 2 Milvus.Client.FieldSchema Name movie_description DataType FloatVector IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension 2 State FieldCreated FieldId 102 EnableDynamicFields False ShardsNum 2 StartPositions (empty)

## Add data to collection

In [11]:
var movieIds = new [] {1L,2L,3L,4L,5L};
var movieNames = new [] {"The Lion King","Inception","Toy Story","Pulp Function","Shrek"};
var movieDescriptions = new ReadOnlyMemory<float>[] { 
        new [] {0.10022575f, -0.23998135f},
        new [] {0.10327095f, 0.2563685f},
        new [] {0.095857024f, -0.201278f},
        new [] {0.106827796f, 0.21676421f},
        new [] {0.09568083f, -0.21177962f}
    };

In [12]:
await collection.InsertAsync(new FieldData[]
{
    FieldData.Create("movie_id", movieIds),
    FieldData.Create("movie_name", movieNames),
    FieldData.CreateFloatVector("movie_description", movieDescriptions)
});

## Persist data

In [13]:
await collection.FlushAsync();

In [14]:
await collection.GetEntityCountAsync()

5

## Search for data

### Create Index

In [15]:
await collection.CreateIndexAsync(
    fieldName: "movie_description", 
    indexType: IndexType.Flat, 
    metricType: SimilarityMetricType.L2, 
    extraParams: new Dictionary<string,string> {["nlist"] = "1024"}, 
    indexName: "movie_idx");

### Load collection

In [16]:
await collection.LoadAsync();
await collection.WaitForCollectionLoadAsync();

### Define search parameters

In [17]:
var parameters = new SearchParameters
{
    OutputFields = { "movie_name" },
    ConsistencyLevel = ConsistencyLevel.Strong,
    ExtraParameters = { ["nprobe"] = "1024" }
};

### Search for data

Search for data using embedding vectors for the query "A movie that's fun for the whole family".

In [18]:
var results = await collection.SearchAsync(
    vectorFieldName: "movie_description",
    vectors: new ReadOnlyMemory<float>[] { new[] {0.12217915f, -0.034832448f } },
    SimilarityMetricType.L2,
    limit: 3,
    parameters);

In [19]:
results

Milvus.Client.SearchResults CollectionName movies FieldsData index value 0 Field: {FieldName: movie_name, DataType: VarChar, Data: 3, RowCount: 3} Data [ Toy Story, Shrek, The Lion King ] RowCount 3 FieldName movie_name FieldId 0 DataType VarChar IsDynamic False Ids Milvus.Client.MilvusIds LongIds [ 3, 5, 1 ] StringIds <null> NumQueries 1 Scores [ 0.028396975, 0.032012466, 0.042568024 ] Limit 3 Limits [ 3 ]

##